In [1]:
import torch
import torch.nn.functional as F

from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATv2Conv
from torch.utils.data import DataLoader, random_split

from torch_geometric.nn.pool import global_mean_pool

In [2]:
from helpers import CVFConfigDataset

In [3]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
# )
dataset = CVFConfigDataset(
    "graph_6b_config_rank_dataset.csv", "graph_6b_edge_index.json", 5
)
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
# data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
        self.gat2 = GATv2Conv(dim_h*heads, dim_h, heads=1)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(h, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        h = F.elu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.MSELoss()
        criterion_sum = torch.nn.MSELoss(reduction='sum')
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        edge_index = dataset.edge_index.t().to(device)
        for epoch in range(1, epochs + 1):
            total_loss = 0
            count = 0
            max_loss = -1
            for _, batch in enumerate(data_loader, 1):
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, edge_index)
                # print("output", out, "y", y)
                loss = criterion_sum(out, y)
                total_loss += loss
                avg_loss = loss/len(batch)
                if avg_loss > max_loss:
                    max_loss = avg_loss
                count += len(batch)
                loss.backward()
                optimizer.step()

            print("Epoch", epoch, "| Loss:", total_loss / count, "| Max Loss:", max_loss)


In [14]:
gnn = VanillaGNN(dataset.num_classes, 128, 1).to(device)
print(gnn)

gnn.fit(train_loader, epochs=25)

VanillaGNN(
  (gat1): GATv2Conv(5, 128, heads=8)
  (gat2): GATv2Conv(1024, 128, heads=1)
  (out): Linear(in_features=128, out_features=1, bias=True)
)


AssertionError: 